# Análise de Cancelamento de Cartão de Crédito

Este notebook tem como objetivo realizar uma análise profunda do dataset para identificar os motivos de cancelamento (Churn) de clientes.

In [13]:
import sys
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

In [14]:
ROOT_DIR = Path().resolve().parent
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))
    
from utils.paths import DATA_DIR
from utils.load_file import load_dataset

## Carregar Dados

In [15]:
file_name = "bank_credit_card_cancellation.csv"

try:
    df = load_dataset(file_name)
except FileNotFoundError:
    raise FileNotFoundError(f"Arquivo não encontrado em {DATA_DIR / file_name}")
except Exception as e:
    raise RuntimeError(f"Erro ao carregar dataset: {e}")

df.head()

,CLIENTNUM,Categoria,Idade,Sexo,Dependentes,Educação,Estado Civil,Faixa Salarial Anual,Categoria Cartão,Meses como Cliente,...,Inatividade 12m,Contatos 12m,Limite,Limite Consumido,Limite Disponível,Mudanças Transacoes_Q4_Q1,Valor Transacoes 12m,Qtde Transacoes 12m,Mudança Qtde Transações_Q4_Q1,Taxa de Utilização Cartão
0,768805383,Cliente,45,M,3,Ensino Médio,Casado,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1335.0,1144,42,1625.0,0.061
1,818770008,Cliente,49,F,5,Ensino Superior,Solteiro,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1541.0,1291,33,3714.0,0.105
2,713982108,Cliente,51,M,3,Ensino Superior,Casado,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2594.0,1887,20,2333.0,0.000
3,769911858,Cliente,40,F,4,Ensino Médio,Não informado,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1405.0,1171,20,2333.0,0.760
4,709106358,Cliente,40,M,3,Sem ensino formal,Casado,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2175.0,816,28,2.5,0.000


In [16]:
df.shape

(10127, 21)

## 1. Limpeza de Dados

In [17]:
# Remover colunas irrelevantes
cols_to_drop = ["CLIENTNUM"]
cols_to_drop += [c for c in df.columns if "Naive_Bayes" in c]

df_clean = df.drop(columns=cols_to_drop, errors="ignore")

print(f"Colunas removidas ({len(cols_to_drop)}): {cols_to_drop}")
print(f"Shape após limpeza: {df_clean.shape}")

Colunas removidas (1): ['CLIENTNUM']
Shape após limpeza: (10127, 20)


In [18]:
# Valores ausentes
missing = df_clean.isnull().sum()
missing = missing[missing > 0]

print("Valores ausentes:")
display(missing if not missing.empty else "Nenhum valor ausente")

# Duplicatas
duplicates = df_clean.duplicated().sum()
print(f"Duplicatas: {duplicates}")

Valores ausentes:


Categoria Cartão    1
dtype: int64

Duplicatas: 0


## 2. Análise Univariada

In [19]:
num_cols = df_clean.select_dtypes(include="number").columns.tolist()
cat_cols = df_clean.select_dtypes(exclude="number").columns.tolist()

print(f"Numéricas ({len(num_cols)}): {num_cols}")
print(f"Categóricas ({len(cat_cols)}): {cat_cols}")

Numéricas (14): ['Idade', 'Dependentes', 'Meses como Cliente', 'Produtos Contratados', 'Inatividade 12m', 'Contatos 12m', 'Limite', 'Limite Consumido', 'Limite Disponível', 'Mudanças Transacoes_Q4_Q1', 'Valor Transacoes 12m', 'Qtde Transacoes 12m', 'Mudança Qtde Transações_Q4_Q1', 'Taxa de Utilização Cartão']
Categóricas (6): ['Categoria', 'Sexo', 'Educação', 'Estado Civil', 'Faixa Salarial Anual', 'Categoria Cartão']


In [20]:
TARGET_COL = "Categoria"

if TARGET_COL not in df_clean.columns:
    raise ValueError(f"Coluna alvo '{TARGET_COL}' não encontrada")

fig = px.pie(
    df_clean,
    names=TARGET_COL,
    title="Distribuição de Cancelamento (Churn)",
    hole=0.4
)
fig.show()

In [21]:
AGE_COL = "Idade"

fig = px.histogram(
    df_clean,
    x=AGE_COL,
    nbins=20,
    title="Distribuição de Idade dos Clientes"
)
fig.show()

## 3. Análise Bivariada (vs Churn)

In [22]:
TRANS_AMT_COL = "Valor Transacoes 12m"

fig = px.box(
    df_clean,
    x=TARGET_COL,
    y=TRANS_AMT_COL,
    title="Valor de Transações (12m) por Status"
)
fig.show()

In [23]:
CONTACTS_COL = "Contatos 12m"

fig = px.histogram(
    df_clean,
    x=CONTACTS_COL,
    color=TARGET_COL,
    barmode="group",
    title="Quantidade de Contatos (12m) vs Churn"
)
fig.show()

## 4. Análise Multivariada (Correlação)

In [24]:
num_cols = df_clean.select_dtypes(include="number").columns

corr_matrix = df_clean[num_cols].corr()

fig = px.imshow(
    corr_matrix,
    aspect="auto",
    color_continuous_scale="RdBu_r",
    title="Mapa de Calor de Correlação"
)
fig.show()

## Resumo de Insights

1. **Contatos**: Clientes com mais contatos (4+) parecem ter maior tendência ao cancelamento.
2. **Transações**: Clientes com menor valor transacionado e menor quantidade de transações são mais propensos ao churn.
3. **Inatividade**: Clientes inativos por mais meses também apresentam maior risco.
